# Declaration

In [1]:
import keras
import re
import nltk
import numpy as np
from keras import preprocessing
from keras import optimizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.models import Model
from keras.layers import SeparableConv1D, MaxPooling1D
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint



keras.__version__

Using TensorFlow backend.


'2.2.4'

# Data Loading

In [2]:
trainFromTextFile = "train.FROM"
trainToTextFile   = "train.TO"
trainFromText     = open(trainFromTextFile, 'r', encoding='utf-8').read().lower()
trainToText       = open(trainToTextFile, 'r', encoding='utf-8').read().lower()
trainFromSentenceTokens = re.split('\n', trainFromText)
trainToSentenceTokens   = re.split('\n', trainToText)
trainFromWordTokens = re.split(' |\n', trainFromText)
trainToWordTokens   = re.split(' |\n', trainToText)

print('Found %s sentences from TrainFrom Text' %len(trainFromSentenceTokens))
print('Found %s sentences from TrainTo Text' %len(trainToSentenceTokens))
print('Found %s words from TrainFrom Text' %len(trainFromWordTokens))
print('Found %s words from TrainTo Text' %len(trainToWordTokens))

does this also apply to the salad?
jesus, i hope not.
Found 29620 sentences from TrainFrom Text
Found 29620 sentences from TrainTo Text
Found 521666 words from TrainFrom Text
Found 479824 words from TrainTo Text


In [41]:
print(trainFromWordTokens[0])
print(trainToWordTokens[0])
print(trainFromSentenceTokens[0])
print(trainToSentenceTokens[0])

does
jesus,
does this also apply to the salad?
jesus, i hope not.


In [3]:
trainInputTokens = trainFromSentenceTokens[0:1000]
trainTargetTokens = trainToSentenceTokens[0:1000]

# Tokenization

In [99]:
max_len = 100    # We will cut comments after 100 words
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizerInput = Tokenizer(num_words=max_words)
tokenizerInput.fit_on_texts(trainInputTokens)

sequencesInput = tokenizerInput.texts_to_sequences(trainInputTokens)
sequencesInput = pad_sequences(sequencesInput, maxlen=max_len)  #Pad so all the arrays are the same size

Inputindex = tokenizerInput.word_index
Inputcount = tokenizerInput.word_counts
nInput = len(tokenizerInput.word_counts) + 1

print("Train From File:\n")
print('Found %s sentences.' %len(trainInputTokens))
print('Found %s sequences.' %len(sequencesInput))
print('Found %s unique tokens.' % len(Inputindex))
print('Found %s unique words.' % len(Inputcount))

Train From File:

Found 1000 sentences.
Found 1000 sequences.
Found 4304 unique tokens.
Found 4304 unique words.


In [107]:
sequencesInput[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,  102,   22,  108, 1505,    3,    1,
       1506])

In [50]:
trainInputTokens[10]

'seems like the wildest and craziest guys are pastors of megachurches.  those guys know how to have fun.  republican senators and representatives are pretty wild too.'

In [51]:
sequencesInput.shape

(1000, 100)

In [102]:
tokenizerTarget = Tokenizer(num_words=max_words)
tokenizerTarget.fit_on_texts(trainTargetTokens)

Targetindex = tokenizerTarget.word_index
Targetcount = tokenizerTarget.word_counts
nTarget     = len(tokenizerTarget.word_counts) + 1

sequencesTarget = tokenizerTarget.texts_to_sequences(trainTargetTokens)
sequencesTarget = pad_sequences(sequencesTarget, maxlen=max_len)  #Pad so all the arrays are the same size

print("Train To File:\n")
print('Found %s sentences.' %len(trainTargetTokens))
print('Found %s sequences.' %len(sequencesTarget))
print('Found %s unique tokens.' % len(Targetindex))
print('Found %s unique words.' % len(Targetcount))

Train To File:

Found 1000 sentences.
Found 1000 sequences.
Found 4129 unique tokens.
Found 4129 unique words.


In [12]:
trainTargetTokens[0]

'jesus, i hope not.'

In [13]:
print(sequencesInput.shape)
print(sequencesTarget.shape)
Targetindex

(1000, 100)
(1000, 100)


{'the': 1,
 'newlinechar': 2,
 'to': 3,
 'a': 4,
 'i': 5,
 'you': 6,
 'and': 7,
 'of': 8,
 'is': 9,
 'that': 10,
 'it': 11,
 'in': 12,
 'for': 13,
 'on': 14,
 'not': 15,
 'they': 16,
 'be': 17,
 'have': 18,
 'was': 19,
 'are': 20,
 'but': 21,
 'if': 22,
 'with': 23,
 "it's": 24,
 'just': 25,
 'my': 26,
 'your': 27,
 'or': 28,
 'like': 29,
 'as': 30,
 'would': 31,
 'all': 32,
 'what': 33,
 'this': 34,
 "don't": 35,
 'so': 36,
 'he': 37,
 'http': 38,
 'think': 39,
 'people': 40,
 'do': 41,
 'me': 42,
 'no': 43,
 'at': 44,
 "'": 45,
 'from': 46,
 'one': 47,
 'can': 48,
 'gt': 49,
 'how': 50,
 'there': 51,
 'about': 52,
 'more': 53,
 'we': 54,
 'get': 55,
 'know': 56,
 'com': 57,
 'some': 58,
 'them': 59,
 'then': 60,
 'an': 61,
 "that's": 62,
 'will': 63,
 'up': 64,
 "i'm": 65,
 'good': 66,
 'well': 67,
 'by': 68,
 'who': 69,
 'because': 70,
 'make': 71,
 'than': 72,
 'their': 73,
 'his': 74,
 'too': 75,
 'when': 76,
 'www': 77,
 'out': 78,
 'why': 79,
 'actually': 80,
 'which': 81,
 'red

In [55]:
sequencesInput[10]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,  243,   28,    1,
       1532,    8, 1533,  244,   14, 1534,    7, 1535,   89,  244,   41,
         51,    3,   18,  922,  923, 1536,    8, 1537,   14,  198, 1538,
         91])

In [60]:
sequencesInput[10, 90]

18

In [68]:
len(Inputindex)

4304

In [109]:
indices = np.arange(sequencesInput)(Inputindex)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [94]:
num_encoder_tokens = len(Inputindex)
num_decoder_tokens = len(Targetindex)
max_encoder_seq_length = max([len(txt) for txt in trainInputTokens])
max_decoder_seq_length = max([len(txt) for txt in trainTargetTokens])

print(num_encoder_tokens)
print(num_decoder_tokens)
print(max_encoder_seq_length)
print(max_decoder_seq_length)

4304
4129
373
913


In [13]:
Targetindex['also']

178

In [16]:
glove_dir = r'C:\Users\mosta\Desktop\Deep Learning Projects\Projects\Lab Exercises\Machine Learning Projects\glove.6B\glove.6B.300d.txt'

embeddings_index = {}
f = open(glove_dir, encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [19]:
# The Embedding layer takes at least two arguments:
# the number of possible tokens(the maximum word index)
# and the dimensionality of the embeddings, here 300.
embedding_dim = 300

embedding_Inputmatrix = np.zeros((max_words, embedding_dim))
for word, i in Inputindex.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_Inputmatrix[i] = embedding_vector

In [20]:
embedding_Targetmatrix = np.zeros((max_words, embedding_dim))
for word, i in Targetindex.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_Targetmatrix[i] = embedding_vector

# Encoder / Decoder

In [25]:
embedding_Inputmatrix.shape

(10000, 300)

In [ ]:
embedding_Inputmatrix[5]

In [8]:
# TODO Fit Data into Encoder / Decoder

In [95]:
encoder_input_data = np.zeros((len(trainInputTokens), max_len, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(trainTargetTokens), max_len, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(trainTargetTokens), max_len, num_decoder_tokens), dtype='float32')

print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(1000, 100, 4304)
(1000, 100, 4129)
(1000, 100, 4129)


In [97]:
encoder_input_data[10, 10, 5]

0.0

In [38]:
trainFromWordTokens[0:5]

['does', 'this', 'also', 'apply', 'to']

In [106]:
for i in trainInputTokens:
    for t in range (0, max_len):
        for word in trainFromSentenceTokens[i]:
            encoder_input_data[i, t, Inputindex[word]] = 1

TypeError: list indices must be integers or slices, not str

In [63]:
newEncoder = np.zeros((len(trainInputTokens), max_len))

In [64]:
newEncoder = sequencesInput[:, :]

In [66]:
newEncoder[10, 90]

18

In [81]:
neEncoder1 = newEncoder[:, :, np.newaxis]

In [82]:
neEncoder1.shape

(1000, 100, 1)

In [90]:
neEncoder1.resize(1000, 100, 4000)

ValueError: cannot resize this array: it does not own its data

In [86]:
neEncoder1.reshape(1000, 100, 4200)

ValueError: cannot reshape array of size 100000 into shape (1000,100,4200)

TypeError: 'numpy.ndarray' object is not callable

In [11]:
for i, (trainInputTokens, trainTargetTokens) in enumerate(zip(trainInputTokens, trainTargetTokens)):
    for t, char in enumerate(trainInputTokens):
        encoder_input_data[i, t, Targetindex[char]] = 1.
    encoder_input_data[i, t + 1:, Targetindex[]] = 1.

KeyError: 2

In [ ]:
latent_dim = 300
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=32,
          epochs=2,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(trainFromWordTokens)])
target_token_index = dict([(char, i) for i, char in enumerate(trainToWordTokens)])

In [ ]:
input_token_index['hello']

In [ ]:
for i, (trainFromWordTokens, trainToWordTokens) in enumerate(zip(trainFromWordTokens, trainToWordTokens)):
    for t, char in enumerate(trainFromWordTokens):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(trainToWordTokens):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [ ]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])